In [ ]:
import arviz as az
from pathlib import Path
from datetime import datetime
import yaml

from estival.sampling import tools as esamptools

import utils as ut
import model as md

parent_output_folder = Path.cwd() / "outputs"

In [2]:
TEST_ANALYSIS_CONFIG = {
    'opti_budget': 100,

    'mcmc_chains': 4,
    'mcmc_cores': 4,
    'mcmc_tune': 100,
    'mcmc_samples': 100,

    'full_runs_burnin': 50,
    'full_runs_samples': 100,
}

FULL_ANALYSIS_CONFIG = {
    'opti_budget': 10000,

    'mcmc_chains': 4,
    'mcmc_cores': 4,
    'mcmc_tune': 1000,
    'mcmc_samples': 5000,

    'full_runs_burnin': 2000,
    'full_runs_samples': 1000,
}

In [3]:
model_config = {
    "start_time": 1850,
    "end_time": 2050,
    "population": 1.e6,
    "seed": 100,   
    "intervention_time": 2025,
}

intervention_params = {
    "transmission_reduction": {
        "rel_reduction": .20
    },
    "preventive_treatment": {
        "rate": .10,
        "efficacy": .8
    },
    "faster_detection": {
        "detection_rate_mutliplier": 2.
    },
    "improved_treatment": {
        "negative_outcomes_rel_reduction": .50
    }
}

# Optimisation

### Find optimal parameter set, varying all parameters, using model with no intervention

In [4]:
model = md.get_tb_model(model_config, intervention_params, active_interventions=[])
mle_params = ut.find_mle(model, opti_budget=analysis_config['opti_budget'])

with open("mle_params.yml", 'w') as file:
    yaml.dump(mle_params, file, default_flow_style=False)

### Check optimal model fit

In [3]:
model.run(ut.default_params | mle_params)
do = model.get_derived_outputs_df()
do['tb_prevalence_per100k'].loc[2010:].plot()
ut.target_data.plot(style='.',color='red')

# Main Analysis 

In [4]:
mle_path = Path.cwd() / "data" / "mle_params.yml"
with open(mle_path, 'r') as file:
    mle_params = yaml.safe_load(file)

In [ ]:
fixed_param = None
analysis_config = TEST_ANALYSIS_CONFIG
analysis_name = 'test' 
folder_path = parent_output_folder / f"{datetime.now().strftime('%Y_%m_%d@%H_%M_%S')}_{analysis_name}_fixed_{fixed_param}"

ut.run_analysis(fixed_param, mle_params, analysis_config, model_config, intervention_params, folder_path, home_path=Path.cwd())